In [1]:
import tensorflow as tf 
from tensorflow import keras 
from keras.layers import Conv2D , MaxPool2D , Flatten , Dense , Dropout , BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical 
from keras.models import Sequential

In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import pandas as pd 
import warnings 
warnings.filterwarnings('ignore')


### here are some of the modules that are used in implementation of the VGG 

In [ ]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input , decode_predictions
from keras.preprocessing.image import load_img , img_to_array 

In [5]:
# donwloading And loading the VGG model 

model = VGG16(weights='imagenet' ,include_top=True , input_shape=(224 , 224, 3))

# displaying the model 
model.summary()


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from numpy import expand_dims


def predict_image(mode , image_path):

    """predicts the class of the given image using the VGG16 model 
     Args: Pretrained VGG 16 model 
      image_path : Top preduiction as a list of class description scores  """
    
    # loading the image with the required target size for VGG16
    img = load_img(image_path , target_size= (224 , 224))

    # convert the iamge to a numpy array 
    image_arr = img_to_array(img)

    # add a batch dimention 
    image_arr = np.expand_dims(image_arr , axis = 0)

    # process the image for VGG16
    image_arr = preprocess_input(image_arr)

    # predict the image class 
    preds = model.predict(image_arr)


    decode_predictions = decode_predictions(preds , top = 5)

    return decode_predictions[0]




In [10]:
from asyncio import streams
import requests 
def download_image(url , save_path):
    try:
        response = requests.get(url , stream = True)
        response.raise_for_status()
        with open(save_path, 'wb') as out_file:
            for chunk in response.iter_content(chunk_size = 8192):
                out_file.write(chunk)
        print(f"image donwloaded")
    except requests.exceptions.RequestException as e:
        print(f"error in downloading image {e}")

image_url = "https://wallpaperaccess.com/full/1485117.jpg"
output_filename = 'cat.jpg'
download_image(image_url , output_filename)




image donwloaded


In [16]:
sample_img_path = download_image(image_url , output_filename)

predicitons = predict_image(model , sample_img_path)

print("preditions")
for i , (imagenet_id ,label , score) in enumerate(predicitons):
    print(f"{i+1} , : {label} , ({score: .2f})")

    

image donwloaded


TypeError: path should be path-like or io.BytesIO, not <class 'NoneType'>